In [1]:
import pandas as pd

file_path = ".\통합데이터_전처리_당뇨고혈압추가.csv"
health_data = pd.read_csv(file_path)

C:\Users\Master\AppData\Local\Temp\ipykernel_1548\1614333348.py:4: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  health_data = pd.read_csv(file_path)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib
import xgboost as xgb

In [3]:
# 데이터 준비 (health_data는 이미 로드된 데이터라고 가정)
X = health_data[['성별', '연령대코드', '신장', '체중', 
                 '수축기 혈압', '이완기 혈압']]
y = health_data['Abdominal_Obesity']

# 범주형 변수 처리 (필요시)
X = pd.get_dummies(X, drop_first=True)

# 결측치 처리
X = X.dropna()
y = y[X.index]

# 데이터 스케일링 (필요시)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 훈련 데이터와 테스트 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [4]:
# XGBoost 모델 정의
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',  # 이진 분류 문제
    eval_metric='logloss',        # 평가 지표
    learning_rate=0.01,           # 학습률
    max_depth=10,                 # 트리 최대 깊이
    n_estimators=200,             # 부스팅 반복 횟수
    scale_pos_weight=10,  # 클래스 불균형 처리
    random_state=42
)

In [6]:
# 학습 과정 모니터링을 위한 설정
eval_set = [(X_train, y_train), (X_test, y_test)]

# 모델 학습
xgb_model.fit(
    X_train,
    y_train,
    eval_metric="logloss",  # 학습 과정에서 모니터링할 평가 지표
    eval_set=eval_set,      # 평가를 위한 데이터셋
    verbose=True            # 학습 과정 출력 (True면 매 단계마다 출력)
)

# 예측
y_pred = xgb_model.predict(X_test)

# 성능 평가
print("정확도:", accuracy_score(y_test, y_pred))
print("분류 보고서:\n", classification_report(y_test, y_pred))

c:\Users\Master\anaconda3\envs\uni\Lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-logloss:0.68945	validation_1-logloss:0.68935
[1]	validation_0-logloss:0.68672	validation_1-logloss:0.68531
[2]	validation_0-logloss:0.68172	validation_1-logloss:0.68165
[3]	validation_0-logloss:0.67768	validation_1-logloss:0.67821
[4]	validation_0-logloss:0.67512	validation_1-logloss:0.67478
[5]	validation_0-logloss:0.67115	validation_1-logloss:0.67131
[6]	validation_0-logloss:0.66715	validation_1-logloss:0.66787
[7]	validation_0-logloss:0.66497	validation_1-logloss:0.66460
[8]	validation_0-logloss:0.66193	validation_1-logloss:0.66129
[9]	validation_0-logloss:0.65818	validation_1-logloss:0.65815
[10]	validation_0-logloss:0.65446	validation_1-logloss:0.65510
[11]	validation_0-logloss:0.65265	validation_1-logloss:0.65211
[12]	validation_0-logloss:0.64971	validation_1-logloss:0.64915
[13]	validation_0-logloss:0.64631	validation_1-logloss:0.64630
[14]	validation_0-logloss:0.64322	validation_1-logloss:0.64352
[15]	validation_0-logloss:0.64104	validation_1-logloss:0.64081
[1

In [7]:
model_save_path = "obesity_model.joblib"
joblib.dump(xgb_model, model_save_path)
print(f"모델이 {model_save_path}에 저장되었습니다.")

모델이 obesity_model.joblib에 저장되었습니다.


추론 시간 + 테스트

In [17]:
print(X_test[:1])

[[-0.93721484 -1.038723   -0.23025953  1.34449692  0.44569232  0.2935656 ]]


In [16]:
import time

# 첫 번째 테스트 데이터에 대해 추론 시간 측정
start_time = time.time()  # 시작 시간 기록

# 첫 번째 샘플에 대해 확률값 예측
single_prediction_prob = xgb_model.predict_proba(X_test[:1])

end_time = time.time()  # 끝 시간 기록

# 추론 시간 계산
inference_time = end_time - start_time
print(f"데이터 하나에 대한 추론 시간: {inference_time:.6f}초")

# 예측 확률 출력
print("클래스 0에 대한 확률:", single_prediction_prob[0][0])
print("클래스 1에 대한 확률:", single_prediction_prob[0][1])

데이터 하나에 대한 추론 시간: 0.001468초
클래스 0에 대한 확률: 0.08330619
클래스 1에 대한 확률: 0.9166938


In [14]:
print(single_prediction)

[1]


In [ ]:
import time
# 사용자로부터 입력된 데이터 (리스트 형태 제거)
input_data = {
    '성별': 1,  # 예: 1은 남성, 0은 여성
    '연령대코드': 2,  # 예: 2는 30대, 1은 20대 등
    '신장': 200.0,  # 예: 170 cm
    '체중': 200.0,  # 예: 65 kg
    '수축기 혈압': 120,  # 예: 120 mmHg
    '이완기 혈압': 80  # 예: 80 mmHg
}

# 입력 데이터를 DataFrame으로 변환 (리스트가 아닌 단일 값 처리)
input_df = pd.DataFrame([input_data])

# 범주형 변수 처리 (pd.get_dummies 사용)
input_df_processed = pd.get_dummies(input_df, drop_first=True)

# 스케일링: 모델에 사용된 X_train과 동일한 방식으로 스케일링
scaler = StandardScaler()
input_scaled = scaler.fit_transform(input_df_processed)

# 첫 번째 테스트 데이터에 대해 추론 시간 측정
start_time = time.time()  # 시작 시간 기록

# XGBoost 모델 예측
prediction = xgb_model.predict(input_scaled)

end_time = time.time()  # 끝 시간 기록

# 추론 시간 계산
inference_time = end_time - start_time
print(f"데이터 하나에 대한 추론 시간: {inference_time}초")

# 예측 결과 출력
print("입력된 데이터:", input_data)
print("예측 결과:", prediction)

데이터 하나에 대한 추론 시간: 0.04007983207702637초
입력된 데이터: {'성별': 1, '연령대코드': 2, '신장': 200.0, '체중': 200.0, '수축기 혈압': 120, '이완기 혈압': 80}
예측 결과: [1]
